In [1]:
import pandas as pd
WaypointNode = pd.read_csv("WaypointNode.csv", usecols=["ID", "Name_lang", "SafeLocID"])
WaypointSafeLocs = pd.read_csv("WaypointSafeLocs.csv")
WaypointEdge = pd.read_csv("WaypointEdge.csv", usecols=["Start", "End", "PlayerConditionID"])

In [2]:
WaypointNode_Loc = (pd.merge(WaypointNode, WaypointSafeLocs,
                           how="inner",
                           left_on="SafeLocID",
                           right_on="ID",
                           suffixes=["", "_WSL"])
                        .drop(columns=["ID_WSL"]))

In [3]:
waypoint_graph = (pd.merge(WaypointEdge, WaypointNode_Loc,
                          how="inner",
                          left_on="Start",
                          right_on="ID",
                          suffixes=["", "_start"])
                     .drop(columns=["SafeLocID", "ID"])
                     .merge(WaypointNode_Loc,
                           how="inner",
                           left_on="End",
                           right_on="ID",
                           suffixes=["", "_end"])
                     .drop(columns=["SafeLocID", "ID"]))

In [4]:
import numpy as np
def distance3d(x1, y1, z1, x2, y2, z2):
    return np.sqrt(np.power(x2 - x1, 2) +
                np.power(y2 - y1, 2) +
                np.power(z2 - z1, 2))

In [5]:
walk_graphs = []
for MapID in WaypointNode_Loc.MapID.unique():
    WaypointsInMap = WaypointNode_Loc[WaypointNode_Loc.MapID == MapID]
    xjoin = pd.merge(WaypointsInMap, WaypointsInMap, how="cross", suffixes=["_start", "_end"])
    xjoin["Distance"] = distance3d(xjoin['Pos[0]_start'], xjoin['Pos[1]_start'], xjoin['Pos[2]_start'],
                                   xjoin['Pos[0]_end'], xjoin['Pos[1]_end'], xjoin['Pos[2]_end'])
    xjoin = xjoin.drop(columns=["SafeLocID_end", "SafeLocID_start"])
    xjoin = (xjoin.rename(columns={
                    "ID_start": "Start",
                    "ID_end": "End"
                })
             .rename(columns=lambda colname: colname.replace("_start", "")))
    walk_graphs.append(xjoin)

In [6]:
graph = pd.concat(walk_graphs + [waypoint_graph], ignore_index=True)
graph["EdgeName"] = graph["Start"].map(str) + " " + graph["End"].map(str)

import dijkstra as dk

class TravelGraph(dk.dijkstra.AbstractDijkstraSPF):
    @staticmethod
    def get_adjacent_nodes(G, u):
        return list(G[G.Start == u].End)
        
    @staticmethod
    def get_edge_weight(G, u, v):
        edgename = str(u) + " " + str(v)
        distance = float(G[G.EdgeName == edgename].Distance.iloc[0])
        if np.isnan(distance):
            return 9999
        return distance

tg = TravelGraph(graph, 292)
path = tg.get_path(298)
pd.DataFrame(path, columns=["ID"]).merge(WaypointNode_Loc, left_on="ID", right_on="ID")

In [14]:
PlayerCondition = pd.read_csv("PlayerCondition.csv", usecols=["ID", "RaceMask"])
ChrRaces = pd.read_csv("ChrRaces.csv", usecols=["ID", "PlayableRaceBit"])
ChrRaces = ChrRaces[ChrRaces["PlayableRaceBit"] != -1] # remove unplayable races

In [15]:
def in_race_bit_mask(bitmask, raceID):
    race_bit = ChrRaces[ChrRaces.ID == raceID].PlayableRaceBit
    return bitmask & (2**race_bit) > 0

In [9]:
waypoint_graph = waypoint_graph.merge(PlayerCondition, how="inner", left_on="PlayerConditionID", right_on="ID").drop(columns=["ID"])

In [10]:
#waypoint_graph = waypoint_graph[waypoint_graph.apply(lambda row: in_race_bit_mask(row.RaceMask, "Human"), axis=1)]

In [19]:
waypoint_graph["CanTraverse"] = waypoint_graph.apply(lambda row: in_race_bit_mask(row.RaceMask, 2), axis=1)
waypoint_graph[waypoint_graph.CanTraverse].drop(columns=["CanTraverse"])

,Start,End,PlayerConditionID,Name_lang,Pos[0],Pos[1],Pos[2],MapID,Name_lang_end,Pos[0]_end,Pos[1]_end,Pos[2]_end,MapID_end,RaceMask
79,109,16,923,Take the portal from Western Earthshrine to Uldum,2039.44000,-4356.23000,98.615600,1,Uldum,-9443.31000,-958.360,111.010000,1,-6184943489809468494
80,99,17,923,Take the portal from Western Earthshrine to Mo...,2042.72000,-4395.74000,98.299100,1,Mount Hyjal,5534.08000,-3624.690,1567.040000,1,-6184943489809468494
81,120,38,923,Take the portal from Pathfinder's Den to Shatt...,1424.09000,-4506.42000,-3.078010,1,"Shattrath, Terrace of Light",-1824.32000,5417.230,-12.430000,530,-6184943489809468494
82,100,101,923,Take the portal from Nordrassil to Orgrimmar,5503.95000,-3624.57000,1567.380000,1,Pathfinder's Den,1445.21000,-4499.560,18.306400,1,-6184943489809468494
83,104,101,923,Take the portal from Temple of Earth to Orgrimmar,990.41900,454.32100,-44.247300,646,Pathfinder's Den,1445.21000,-4499.560,18.306400,1,-6184943489809468494
84,117,101,923,Take the portal from Sunfury Spire to Orgrimmar,10003.30000,-7109.87000,47.705400,530,Pathfinder's Den,1445.21000,-4499.560,18.306400,1,-6184943489809468494
85,121,101,923,Take the portal from Terrace of Light to Orgri...,-1899.68000,5392.73000,-12.426400,530,Pathfinder's Den,1445.21000,-4499.560,18.306400,1,-6184943489809468494
86,126,101,923,Take the portal from Tower of Elements to Orgr...,5266.40000,-4075.89000,21.130200,1116,Pathfinder's Den,1445.21000,-4499.560,18.306400,1,-6184943489809468494
87,130,101,923,Take the portal from Crumbled Palace to Orgrimmar,-8.19146,6756.28000,53.834100,1220,Pathfinder's Den,1445.21000,-4499.560,18.306400,1,-6184943489809468494
88,135,101,923,Take the portal from Hall of Ancient Paths to ...,-1123.95000,759.76000,433.623000,1642,Pathfinder's Den,1445.21000,-4499.560,18.306400,1,-6184943489809468494
